In [ ]:
from tune_util import *
import numpy as np
import pickle
import cv2

In [ ]:
import random
import configparser
import os
episodes_config = configparser.ConfigParser(
    allow_no_value=True, inline_comment_prefixes=";"
)
episodes_config.read(os.path.join(os.getcwd(), "params/episode_params_val.ini"))
series = episodes_config.sections()[1:]

In [ ]:
def_num = np.array([10,1,3,1000,4,1,0.8,2,0.35,2,3])
def_num_log = np.log(def_num)

In [ ]:
my_log = []
res = exec_seqs(def_num_log,'test_eval',['t_ETH1','--render'],log_file=my_log)
res

In [ ]:
import cma

In [ ]:
es = cma.CMAEvolutionStrategy(def_num_log,0.1)
opt_iter = 0
best_subsets = []

while not es.stop():
    opt_iter += 1

    solutions = es.ask()
    #r_vals = []
    chosen = [random.choice(series),'--render']
    chosen = ['t_ETH1','--render']

    for sol_i,params in enumerate(solutions):
        res = exec_seqs(params,'mancma_{:03d}'.format(sol_i),chosen,log_file=my_log)
        #r_vals.append(res)
    
    ind_r = []
    for i in sorted(glob.glob(os.path.join(os.getcwd(),'tests/socnav/mancma_*'))):
        # scary what if multiple run!?!
        for pkl in glob.glob(os.path.join(i,'*/*.pkl')):
            with open(pkl,'rb') as fp:
                res = pickle.load(fp)

            success = res['success']
            time_taken = res['total_sim_time_taken']
            collisions = res['total_collisions']
            robot_motion = res['robot_motion_energy']
            ped_dist = np.exp(-res['closest_pedestrian_distance']*2).sum()
            costs = [5.1-5*success,time_taken,collisions+1,robot_motion,ped_dist]
            cost = np.prod(costs)**(1.0/len(costs))
            print(i,cost)

            jpgs = sorted(glob.glob(pkl.replace('episode_score_','').replace('.pkl','_obs_obs*.jpg')))
            read_data = [cv2.imread(_) for _ in jpgs]
            ind_r.append((cost,read_data))

    def compare(i,j):
        cv2.destroyAllWindows()

        c1,set1 = ind_r[i]
        c2,set2 = ind_r[j]

        s12_s = list(set1[0].shape)
        s12_s[1]*=2
        while True:
            for i in range(max(len(set1),len(set2))):
                cv2.imshow("Comp {:.1f} {:.1f}".format(c1,c2), 
                           np.stack([set1[min(i,len(set1)-1)],set2[min(i,len(set2)-1)]],axis=1).reshape(s12_s))
                a = cv2.waitKey(30)

                if a == ord('1'):
                    return 1
                if a == ord('2'):
                    return -1
                if a == ord('3'):
                    return 2*(c1 >c2)-1
                if a == ord('q'):
                    return
                else:
                    continue
    res = sorted(list(range(len(ind_r))),key=functools.cmp_to_key(compare))
    best_subsets.append((solutions[res[-1]],ind_r[res[-1]]))
    t_vals = np.arange(len(solutions))/len(solutions)-opt_iter
    t_vals = t_vals[::-1] # rewards become costs
    r_vals = []
    for i in sorted(range(len(solutions))):
        v = t_vals[res.index(i)]
    es.tell(solutions, r_vals)
    es.logger.add()
    es.disp()

In [ ]:
results = {'cfg':es.best.x,'func':es.best.f,'log':my_log}
import glob
import pickle
N=len(glob.glob('mancma_results*.pkl'))
with open('mancma_results_{}.pkl'.format(N),'wb') as fp:
    pickle.dump(results,fp)

In [ ]:
chosen = [random.choice(series)]
res1 = exec_seqs(def_num_log,'test_eval',chosen)
res2 = exec_seqs(es.best.x,'test_eval',chosen)

res1,res2